# Configure your Jupyter Environment

The following notebook can be used to configure your Jupyter environment.  The product of running this notebook are the following:

- `requirements.txt`: Used to upgrade/downgrade your environment with the FABLIB version used by these example notebooks.
- `fabric_rc`:  File used to configure a FABlib application.  
- `ssh_config`: File used to ssh from from a terminal to FABRIC VM by jumping through the FABRIC bastion host. 

## Copy your Bastion Key to your JupyterLab Environment

In order to minimize security incidents on FABRIC, access to VMs and other resources administered by users is controlled using a bastion host. You will need to set up an ssh keypair that will be used to jump through the bastion host to your VMs and other resources. This keypair is unique to you and is only used to set up ssh proxy connections through the bastion host to your FABRIC resources. 

The following notebook walks through creating and submitting a bastion keypair necessary to use FABRIC. More information about how to access your experiment through the bastion host can be found [here](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/).

If you have not created a bastion host keypair, create a new one in the [Experiments](https://portal.fabric-testbed.net/experiments#sshKeys) section of the FABRIC Portal.

Copy your bastion private key to your Jupyter container. Drag/drop your key from your local machine to the file browser in your Juypyter browser window. Note the name of your key and location where you put it.

## Set Bastion Username and Project ID

Edit the following cell by entering your FABRIC bastion username and the correct path to the copy of your bastion private key in your Jupyter container.  

- Your bastion user name can be found on your [user profile page](https://portal.fabric-testbed.net/user) in the FABRIC portal (click "My SSH Keys").
- The path to your bastion key was determined when you copied it to the Jupyter container.  

The Project ID can be from any of your projects. The ID can be found on the [projects](https://portal.fabric-testbed.net/projects) tab in the FABRIC portal.

In [ ]:
export FABRIC_BASTION_USERNAME=<BASTION_USERNAME>
export FABRIC_PROJECT_ID=<PROJECT_ID>

export FABRIC_BASTION_PRIVATE_KEY_LOCATION=${HOME}/work/fabric_config/fabric_bastion_key

export FABRIC_BASTION_SSH_CONFIG_FILE=${HOME}'/work/fabric_config/ssh_config'
export FABRIC_RC_FILE=${HOME}'/work/fabric_config/fabric_rc'

## Set other Tokens and Keys

The the path to where you put your bastion key.The defaults below should work within the Jupyter environment hosted by FABRIC.  Custom installations will require modifications to this configuration.


In [ ]:
export FABRIC_TOKEN_FILE=${HOME}'/.tokens.json'
export FABRIC_SLICE_PRIVATE_KEY_FILE=${HOME}/work/fabric_config/slice_key
export FABRIC_SLICE_PUBLIC_KEY_FILE=${FABRIC_SLICE_PRIVATE_KEY_FILE}.pub

Optionally, create a new slice keypair. 

In [ ]:
#ssh-keygen -t rsa -b 3072 -f $FABRIC_SLICE_PRIVATE_KEY_FILE -q -N ""



Set the Permissions on your bastion key and private slice key. Your private bastion key must not have it permissions too open.

In [ ]:
chmod 600 ${FABRIC_BASTION_PRIVATE_KEY_LOCATION}
chmod 600 ${FABRIC_SLICE_PRIVATE_KEY_FILE}

## FABLIB Version

This package of example notebooks was created using the version of fabrictestbed-extensions (i.e. the FABLIB package) specified in the local `requirements.txt` file.  

In [5]:
cat requirements.txt

fabrictestbed-extensions==1.3.3b3


These examples may work with your currently installed default FABLIB version.  However, you may want to upgrade/downgrade the FABLIB version to the one used to develop these examples.  The following cell will synchronize your current FABLIB version with these examples.

In [7]:
pip install -r requirements.txt --user --quiet

Your Jupyter container will often be updated with the current default FABLIB version.  You can permanently override these updates by setting your desired version in
`~/work/fabric_config/requirements.txt`.  If you would like to permanently configure your Jupyter environment to use the FABLIB version used for this set of examples, copy the local `requirnments.txt` file to `~/work/fabric_config/requirements.txt` using the following command.

In [ ]:
cp requirements.txt ~/work/fabric_config/requirements.txt

## Create the FABRIC Config File 


The following cell creates the `fabric_rc` file using the values configured above. 


In [ ]:
cat <<EOF > ${FABRIC_RC_FILE}
export FABRIC_CREDMGR_HOST=cm.fabric-testbed.net
export FABRIC_ORCHESTRATOR_HOST=orchestrator.fabric-testbed.net

export FABRIC_PROJECT_ID=${FABRIC_PROJECT_ID}
export FABRIC_TOKEN_LOCATION=${FABRIC_TOKEN_FILE}

export FABRIC_BASTION_HOST=bastion-1.fabric-testbed.net
export FABRIC_BASTION_USERNAME=${FABRIC_BASTION_USERNAME}

export FABRIC_BASTION_KEY_LOCATION=${FABRIC_BASTION_PRIVATE_KEY_LOCATION}
#export FABRIC_BASTION_KEY_PASSPHRASE=

export FABRIC_SLICE_PRIVATE_KEY_FILE=${FABRIC_SLICE_PRIVATE_KEY_FILE}
export FABRIC_SLICE_PUBLIC_KEY_FILE=${FABRIC_SLICE_PUBLIC_KEY_FILE} 
#export FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE=

export FABRIC_LOG_FILE=${HOME}/fablib.log
export FABRIC_LOG_LEVEL=INFO 

export FABRIC_AVOID=''

export FABRIC_SSH_COMMAND_LINE="ssh -i {{ _self_.private_ssh_key_file }} -F /home/fabric/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }}"
EOF

## Create the SSH Config File

The following cell creates an SSH config file using the values configure above.  The config file allows you to use ssh to jump through the bastion host from a terminal command line.  After creating this file, you can ssh and pass the file with the `-F` option.

Alternatively, you can directly edit your [ssh config](~/fabric_config/ssh_config) file.


In [ ]:
cat <<EOF > ${FABRIC_BASTION_SSH_CONFIG_FILE}
UserKnownHostsFile /dev/null
StrictHostKeyChecking no
ServerAliveInterval 120 

Host bastion-?.fabric-testbed.net
     User ${FABRIC_BASTION_USERNAME}
     ForwardAgent yes
     Hostname %h
     IdentityFile ${FABRIC_BASTION_PRIVATE_KEY_LOCATION}
     IdentitiesOnly yes

Host * !bastion-?.fabric-testbed.net
     ProxyJump ${FABRIC_BASTION_USERNAME}@bastion-1.fabric-testbed.net:22
EOF